In [9]:
from google.colab import files
files.upload()

Saving c5d762505b04974094e6fc433d953abfd3bd8c83_00.jpg to c5d762505b04974094e6fc433d953abfd3bd8c83_00.jpg


{'c5d762505b04974094e6fc433d953abfd3bd8c83_00.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc0\x00\x11\x08\x01\xa2\x02\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x02\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x05\x03\x06\x00\x02\x07\x01\x08\xff\xc4\x00G\x10\x00\x02\x01\x03\x02\x03\x05\x04\x07\x05\x06\x05\x04\x03\x01\x01\x01\x02\x03\x00\x04\x11\x05!\x121A\x06\x13Qaq"2\x81\x91\x07\x14BR\xa1\xb1\xc1\x15#3b\xd1$Cr\x82\xe1\xf04Sc\x92\xa2\x16%5\xb2s\xc2\xf1\x08d\xff\xc4\x00\x1b\x01\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x02\x05\x06\x01\x00\x07\xff\xc4\x008\x11\x00\x01\x04\x01\x

###Data Augmentation

In [11]:
from keras.preprocessing.image import ImageDataGenerator , array_to_img,img_to_array
from keras.preprocessing import image
datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

img = image.load_img('1.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i=0
for batch in datagen.flow(x,batch_size=1,save_to_dir='sample_data',save_prefix='Sahil',save_format='jpeg'):
  i += 1
  if i>20:
    break

In [1]:
!pip install keras_tuner

     |████████████████████████████████| 71kB 10.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=e6e379899d40eda41a851ba28e1d95892ec30b87b7dbdac6c7ed2ec44ea88747
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=a966deb7ec747b1ab4bbd4edde8e33d587ba0b11b598c5c69c03c4964452116a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_minst = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_minst.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [7]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1) ### 1 for grey scale

###Keras Tuner

In [14]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size = hp.Choice('conv_1_kernel',values=[3,5]),
                                activation = 'relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size = hp.Choice('conv_1_kernel',values=[3,5]),
                                activation = 'relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation = 'relu'
                            ),
                            keras.layers.Dense(10,activation='relu')

  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters 

In [18]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Minst_Fashion')

In [20]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 6 Complete [00h 00m 19s]
val_accuracy: 0.10499999672174454

Best val_accuracy So Far: 0.1720000058412552
Total elapsed time: 00h 02m 21s
INFO:tensorflow:Oracle triggered exit


In [21]:
model = tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 96)        230496    
_________________________________________________________________
flatten (Flatten)            (None, 38400)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                3072080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,305,882
Trainable params: 3,305,882
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 5.3373 - accuracy: 0.1764 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 5.3440 - accuracy: 0.1770 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 6/10
1688/1688 [==============================] - 9s 6ms/step - loss: 5.3336 - accuracy: 0.1738 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 5.3130 - accuracy: 0.1781 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 8/10
1688/1688 [==============================] - 9s 6ms/step - loss: 5.3203 - accuracy: 0.1763 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 9/10
1688/1688 [==============================] - 9s 6ms/step - loss: 5.3226 - accuracy: 0.1792 - val_loss: 5.1849 - val_accuracy: 0.1720
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 5.3413 - accuracy: 0.1774 - val_loss: 5.1849 - val_accurac

###Transfer Learning

In [24]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
Image_Size = [224,224]
train_path = 'Data/train'
test_path = 'Data/test'

In [ ]:
vgg = VGG16(input_shape = Image_Size + [3],weights='imagenet',include_top=False)

for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('Data/train/*')

x = Flatten()(vgg.output)
pred = Dense(len(folders),activation='softmax')(x)
model = Model(inputs=vgg.input,outputs = pred)
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',optimizer='Adam',metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')